In [1]:
# !git clone https://github.com/hussien/cifar10_pytorch.git

In [2]:
# ! rm -r '/content/cifar10_pytorch'

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [4]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

import os
import torch
import torchvision
import tarfile
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt

In [5]:
!pip install yacs

In [6]:
import os
import argparse
import torch
import os, sys
from cifar10_pytorch.cifar10_pytorch.engine import train, test
from cifar10_pytorch.cifar10_pytorch.config import cfg
from cifar10_pytorch.cifar10_pytorch.data import dataset
from cifar10_pytorch.cifar10_pytorch.modeling import build_model

In [7]:
# ! rm -r 'CINIC-10-Filtered_1K'

In [8]:
# !unzip -q  '/content/cifar10_pytorch/CINIC-10-Filtered_1K.zip' -d '/content/'
# !unzip -q  '/content/cifar10_pytorch/train.zip' -d '/content/'
# !unzip -q  '/content/cifar10_pytorch/test.zip' -d '/content/'

In [9]:
def arg_parser():
    parser = argparse.ArgumentParser(description="CIFAR10 training")
    parser.add_argument(
        "--config",
        default='/content/cifar10_pytorch/configs/efficientnetB0.yaml',
        # default='/content/cifar10_pytorch/configs/efficientnetB4.yaml',
        help="path to config file",
        type=str
    )
    parser.add_argument(
        '--tfboard', help='tensorboard path for logging', type=str, default='out')
    parser.add_argument('--checkpoint_dir', type=str,
                        default='checkpoints',
                        help='directory where checkpoint files are saved')
    parser.add_argument('--resume', type=str,
                        default=None,
                        help='checkpoint file path')
    return parser.parse_args("")

In [10]:
!pip install tensorboardX

In [ ]:
# args and configs
args = arg_parser()
if args.config:
    cfg.merge_from_file(args.config)
cfg.freeze()
os.makedirs(args.checkpoint_dir, exist_ok=True)

# Model definition
# cfg.SOLVER.BATCHSIZE=64
model = build_model(cfg)
# cfg.MODEL.DEVICE='cpu'
print(cfg.MODEL.DEVICE)
device = cfg.MODEL.DEVICE
model.to(device)
# Build dataloader
dataloader_train, dataloader_test = dataset.prepare_cifar10_dataset(cfg)
print(len(dataloader_train.dataset))

# Optimizer settings
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=cfg.SOLVER.BASE_LR,
                            momentum=cfg.SOLVER.MOMENTUM,
                            weight_decay=cfg.SOLVER.WEIGHT_DECAY,
                            )

scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer,
    cfg.SOLVER.MILESTONES,
    cfg.SOLVER.GAMMA,
)
if args.resume:
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    #scheduler.last_epoch = checkpoint['epoch']
    start_epoch = checkpoint['epoch']
else:
    start_epoch = 0

# Tensorboard settings
if args.tfboard:
    print("using tfboard")
    from tensorboardX import SummaryWriter
    tblogger = SummaryWriter(args.tfboard)
else:
    tblogger = None

# Training Loop
# end_epoch = cfg.SOLVER.END_EPOCH
end_epoch = 30
train_loss_epoch,test_loss_epoch=[],[]
train_acc_epoch,test_acc_epoch=[],[]
for epoch in range(start_epoch, end_epoch):
    scheduler.step()
    print("------ start epoch {} / {} with lr = {:.5f} -----".format(
        epoch+1, end_epoch, scheduler.get_lr()[0])
    )
    
    train_loss, train_acc = train(model, device, dataloader_train, criterion, optimizer, epoch)
    train_loss_epoch.append(train_loss)
    train_acc_epoch.append(train_acc)
    if epoch%5 == 0:
      test_loss, test_acc = test(model, device, dataloader_test, criterion, epoch)
    test_loss_epoch.append(test_loss)
    test_acc_epoch.append(test_acc)
    if tblogger:
        tblogger.add_scalar('train/loss', train_loss, epoch+1)
        tblogger.add_scalar('train/acc', train_acc, epoch+1)
        tblogger.add_scalar('test/acc', test_acc, epoch+1)
    torch.save({'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                },
                os.path.join(args.checkpoint_dir, "snapshot.ckpt"))

cuda
Files already downloaded and verified
10100
using tfboard


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:417: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


------ start epoch 1 / 30 with lr = 0.01500 -----

train set size= 158  batch size= 64
train-> epoch 1, iter 100 / 158, Loss: 1.164 | Acc: 61.750

test set size= 157  batch size= 64
test-> epoch 1, iter 100 / 157, Loss: 0.843 | Acc: 70.938
Accuracy: 70.5500
------ start epoch 2 / 30 with lr = 0.01500 -----

train set size= 158  batch size= 64
train-> epoch 2, iter 100 / 158, Loss: 0.500 | Acc: 82.047
------ start epoch 3 / 30 with lr = 0.01500 -----

train set size= 158  batch size= 64
train-> epoch 3, iter 100 / 158, Loss: 0.363 | Acc: 87.266
------ start epoch 4 / 30 with lr = 0.01500 -----

train set size= 158  batch size= 64
train-> epoch 4, iter 100 / 158, Loss: 0.278 | Acc: 90.469
------ start epoch 5 / 30 with lr = 0.01500 -----

train set size= 158  batch size= 64
train-> epoch 5, iter 100 / 158, Loss: 0.214 | Acc: 92.391
------ start epoch 6 / 30 with lr = 0.01500 -----

train set size= 158  batch size= 64
train-> epoch 6, iter 100 / 158, Loss: 0.169 | Acc: 94.234

test set si

In [ ]:
import datetime 
currentDT = datetime.datetime.now()
torch.save({'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss_epoch': train_loss_epoch,
            'test_loss_epoch': test_loss_epoch,
            'train_acc_epoch': train_acc_epoch,
            'test_acc_epoch': test_acc_epoch
            },
            os.path.join("/content/", "EfficientNetB0_CINIC10_30epoch_"+str(currentDT)+".ckpt"))

In [ ]:
model.eval()
test_loss = 0
correct = 0
total = 0
with torch.no_grad():
        for batch_i, (inputs, targets) in enumerate(dataloader_test):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            if batch_i > 0 and (batch_i + 1) % 100 == 0:
                print("test-> epoch {}, iter {} / {}, Loss: {:.3f} | Acc: {:.3f}".format(
                    epoch+1,
                    batch_i+1,
                    len(dataloader_test),
                    test_loss/(batch_i+1),
                    100.*correct/total,
                ))
acc = 100.*correct/total
print("Accuracy: {:.4f}".format(acc))

In [ ]:
inputs.detach()
targets.detach()

In [ ]:
# loss_train = history.history['train_loss']
# loss_val = history.history['val_loss']
loss_train=train_loss_epoch
loss_test=test_loss_epoch
epochs = range(1,len(train_loss_epoch)+1)
plt.plot(epochs, loss_train, 'b', label='CINIC Train loss')
plt.plot(epochs, loss_test, 'g', label='MNIST Test loss')
plt.title('Training and Test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

epochs = range(1,len(train_acc_epoch)+1)
plt.plot(epochs, train_acc_epoch, 'b', label='CINIC Train Acc')
plt.plot(epochs, test_acc_epoch, 'g', label='MNIST Test Acc')
plt.title('Training and test Acc')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# !unzip -q  '/content/gdrive/MyDrive/DLCourseProject/train.zip' -d '/content/train'
!unzip -q  '/content/gdrive/MyDrive/DL_Course/test.zip' -d '/content/test'

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
from cifar10_pytorch.cifar10_pytorch.data import transform
validation_img_paths = []
transform_test = transform.build_transforms(cfg, is_train=False)
for i in range(1,51):
  validation_img_paths.append("/content/test/all/"+str(i).zfill(4)+".png")
img_list = [Image.open( img_path) for img_path in validation_img_paths]

validation_batch = torch.stack([transform_test(img).to(device)
                                for img in img_list])

pred_logits_tensor = model(validation_batch)
pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()
classes=['airplane','automobile','bird', 'cat','deer', 'dog', 'frog','horse', 'ship',  'truck']
fig, axs = plt.subplots(10, 5, figsize=(10, 20))
for i, img in enumerate(img_list):
    ax = axs[int(i/5),i%5]
    ax.axis('off')
    # print(pred_probs[i])
    ax.set_title(str(np.argmax(pred_probs[i]))+" "+classes[np.argmax(pred_probs[i])])                                                        
    ax.imshow(img)